# Open Channel Flow Calculator - Google Colab Version

This notebook contains a complete open channel flow calculator for hydraulic engineering.

**Features:**
- Normal depth and critical depth calculations (Manning's & Chezy equations)
- Multiple channel geometries (rectangular, trapezoidal, circular, triangular, compound, wide)
- Hydraulic jump analysis
- Gradually varied flow (GVF) profiles
- Weir and sluice gate calculations
- Flow regime classification (subcritical, critical, supercritical)
- Compound channels for rivers with floodplains

**Author:** Priscilla  
**Date:** August 2024

---

## 1. Installation and Setup

First, install required dependencies:

In [1]:
# Install dependencies (only needed on first run)
!pip install numpy scipy -q

import math
import numpy as np
from scipy.optimize import fsolve, brentq

print("Dependencies installed successfully!")
print(f"NumPy version: {np.__version__}")

Dependencies installed successfully!
NumPy version: 2.0.2


## 2. Physical Constants

In [2]:
# Physical constants
g = 9.81  # Acceleration due to gravity (m/s²)

## 3. Channel Geometry Classes

These classes define different channel cross-sections and provide methods to calculate geometric properties.

In [3]:
class ChannelSection:
    """Base class for channel cross-sections"""

    def area(self, depth):
        """Calculate flow area for given depth"""
        raise NotImplementedError

    def wetted_perimeter(self, depth):
        """Calculate wetted perimeter for given depth"""
        raise NotImplementedError

    def hydraulic_radius(self, depth):
        """Calculate hydraulic radius R = A/P"""
        A = self.area(depth)
        P = self.wetted_perimeter(depth)
        return A / P if P > 0 else 0

    def top_width(self, depth):
        """Calculate top width of water surface"""
        raise NotImplementedError

    def hydraulic_depth(self, depth):
        """Calculate hydraulic depth D = A/T"""
        A = self.area(depth)
        T = self.top_width(depth)
        return A / T if T > 0 else 0


class RectangularChannel(ChannelSection):
    """Rectangular channel cross-section"""

    def __init__(self, width):
        self.width = width

    def area(self, depth):
        return self.width * depth

    def wetted_perimeter(self, depth):
        return self.width + 2 * depth

    def top_width(self, depth):
        return self.width


class TrapezoidalChannel(ChannelSection):
    """Trapezoidal channel cross-section"""

    def __init__(self, bottom_width, side_slope):
        """
        Parameters:
        - bottom_width: bottom width (m)
        - side_slope: side slope (horizontal:vertical), e.g., 1.5 means 1.5H:1V
        """
        self.bottom_width = bottom_width
        self.side_slope = side_slope

    def area(self, depth):
        return (self.bottom_width + self.side_slope * depth) * depth

    def wetted_perimeter(self, depth):
        return self.bottom_width + 2 * depth * math.sqrt(1 + self.side_slope**2)

    def top_width(self, depth):
        return self.bottom_width + 2 * self.side_slope * depth


class CircularChannel(ChannelSection):
    """Circular channel cross-section (pipe)"""

    def __init__(self, diameter):
        self.diameter = diameter
        self.radius = diameter / 2

    def _theta(self, depth):
        """Calculate central angle from depth"""
        y = depth
        R = self.radius
        if y <= 0:
            return 0
        if y >= 2 * R:
            return math.pi
        cos_theta = 1 - y / R
        cos_theta = max(-1, min(1, cos_theta))
        return math.acos(cos_theta)

    def area(self, depth):
        theta = self._theta(depth)
        R = self.radius
        return R**2 * (theta - math.sin(theta) * math.cos(theta))

    def wetted_perimeter(self, depth):
        theta = self._theta(depth)
        return 2 * self.radius * theta

    def top_width(self, depth):
        theta = self._theta(depth)
        return 2 * self.radius * math.sin(theta)


class TriangularChannel(ChannelSection):
    """V-shaped triangular channel"""

    def __init__(self, side_slope=None, semi_angle=None):
        """
        Parameters:
        - side_slope: side slope (horizontal:vertical)
        - semi_angle: semi-angle from vertical in degrees
        """
        if side_slope is not None:
            self.side_slope = side_slope
        elif semi_angle is not None:
            self.side_slope = math.tan(math.radians(semi_angle))
        else:
            raise ValueError("Must provide either side_slope or semi_angle")

    def area(self, depth):
        return self.side_slope * depth**2

    def wetted_perimeter(self, depth):
        return 2 * depth * math.sqrt(1 + self.side_slope**2)

    def top_width(self, depth):
        return 2 * self.side_slope * depth


class CompoundChannel(ChannelSection):
    """Compound channel with main section and different geometry below a breakpoint"""

    def __init__(self, bottom_section_type, bottom_params, break_depth,
                 top_section_type=None, top_params=None):
        """
        Parameters:
        - bottom_section_type: Type of bottom section ('rectangular', 'trapezoidal', 'triangular', 'circular')
        - bottom_params: Parameters for bottom section (dict)
        - break_depth: Depth at which geometry changes (m)
        - top_section_type: Type of top section (if None, uses same as bottom)
        - top_params: Parameters for top section (dict)

        Example:
        # River with rectangular main channel and trapezoidal floodplains
        channel = CompoundChannel(
            bottom_section_type='rectangular',
            bottom_params={'width': 10.0},
            break_depth=2.0,
            top_section_type='trapezoidal',
            top_params={'bottom_width': 10.0, 'side_slope': 5.0}
        )
        """
        self.bottom_section = self._create_section(bottom_section_type, bottom_params)
        self.break_depth = break_depth

        if top_section_type is not None and top_params is not None:
            self.top_section = self._create_section(top_section_type, top_params)
        else:
            self.top_section = None

    def _create_section(self, section_type, params):
        """Factory method to create channel section"""
        if section_type == 'rectangular':
            return RectangularChannel(params['width'])
        elif section_type == 'trapezoidal':
            return TrapezoidalChannel(params['bottom_width'], params['side_slope'])
        elif section_type == 'triangular':
            if 'side_slope' in params:
                return TriangularChannel(side_slope=params['side_slope'])
            else:
                return TriangularChannel(semi_angle=params['semi_angle'])
        elif section_type == 'circular':
            return CircularChannel(params['diameter'])
        else:
            raise ValueError(f"Unknown section type: {section_type}")

    def area(self, depth):
        """Calculate area for compound section"""
        if depth <= self.break_depth:
            return self.bottom_section.area(depth)
        else:
            A_bottom = self.bottom_section.area(self.break_depth)
            if self.top_section is not None:
                A_top = self.top_section.area(depth - self.break_depth)
            else:
                A_top = self.bottom_section.area(depth) - A_bottom
            return A_bottom + A_top

    def wetted_perimeter(self, depth):
        """Calculate wetted perimeter for compound section"""
        if depth <= self.break_depth:
            return self.bottom_section.wetted_perimeter(depth)
        else:
            # For compound sections, wetted perimeter calculation depends on geometry
            # This is a simplified version
            if self.top_section is not None:
                P_bottom = self.bottom_section.wetted_perimeter(self.break_depth)
                P_top = self.top_section.wetted_perimeter(depth - self.break_depth)
                # Subtract the interface width that's counted twice
                T_break = self.bottom_section.top_width(self.break_depth)
                return P_bottom + P_top - T_break
            else:
                return self.bottom_section.wetted_perimeter(depth)

    def top_width(self, depth):
        """Calculate top width for compound section"""
        if depth <= self.break_depth:
            return self.bottom_section.top_width(depth)
        else:
            if self.top_section is not None:
                return self.top_section.top_width(depth - self.break_depth)
            else:
                return self.bottom_section.top_width(depth)


class WideChannel(RectangularChannel):
    """Wide rectangular channel (unit width analysis)"""

    def __init__(self):
        super().__init__(width=1.0)

    def hydraulic_radius(self, depth):
        """For wide channels, R ≈ y"""
        return depth


print("Channel geometry classes loaded successfully!")

Channel geometry classes loaded successfully!


In [4]:
class SemiCircularChannel(ChannelSection):
    """Semi-circular channel cross-section (always half-filled circular section)"""

    def __init__(self, radius):
        self.radius = radius

    def area(self, depth):
        """For semi-circular channel, area at depth y"""
        R = self.radius
        if depth <= 0:
            return 0
        if depth >= 2*R:
            return math.pi * R**2 / 2  # Half circle area

        # For depth < diameter, use circular segment formula
        # Using chord height h from bottom
        h = depth
        if h > R:
            # Upper part - calculate as full semicircle minus top segment
            h_top = 2*R - h  # height of air space from top
            theta_top = 2 * math.acos(h_top / R)
            A_top = (R**2 / 2) * (theta_top - math.sin(theta_top))
            return (math.pi * R**2 / 2) - A_top
        else:
            # Lower part - direct circular segment
            theta = 2 * math.acos((R - h) / R)
            return (R**2 / 2) * (theta - math.sin(theta))

    def wetted_perimeter(self, depth):
        """Wetted perimeter for semi-circular channel"""
        R = self.radius
        if depth <= 0:
            return 0
        if depth >= 2*R:
            return math.pi * R  # Half circle perimeter

        # Calculate arc length based on depth
        h = depth
        if h > R:
            h_top = 2*R - h
            theta_top = 2 * math.acos(h_top / R)
            return math.pi * R - R * theta_top
        else:
            theta = 2 * math.acos((R - h) / R)
            return R * theta

    def top_width(self, depth):
        """Top width for semi-circular channel"""
        R = self.radius
        if depth <= 0:
            return 0
        if depth >= 2*R:
            return 0  # At full depth, top width is zero (closed at top)

        # Calculate chord length
        h = depth
        if h > R:
            h_top = 2*R - h
            return 2 * math.sqrt(R**2 - h_top**2)
        else:
            y_from_center = R - h
            return 2 * math.sqrt(R**2 - y_from_center**2)


print("SemiCircularChannel class added!")

SemiCircularChannel class added!


## 4. Flow Calculations

Functions for calculating normal depth, critical depth, Froude number, and flow regimes.

In [5]:
# ===== CHÉZY EQUATION FUNCTIONS =====

def chezy_velocity(R, S, C):
    """Calculate velocity using Chézy equation: V = C * sqrt(R * S)

    Parameters:
    - R: hydraulic radius (m)
    - S: slope (m/m)
    - C: Chézy coefficient (m^0.5/s)
    """
    return C * math.sqrt(R * S)


def chezy_discharge(channel, depth, S, C):
    """Calculate discharge using Chézy equation"""
    A = channel.area(depth)
    R = channel.hydraulic_radius(depth)
    V = chezy_velocity(R, S, C)
    return A * V


def normal_depth_chezy(channel, Q, S, C, initial_guess=1.0, max_depth=20.0):
    """Calculate normal depth using Chézy equation"""
    def equation(y):
        if y <= 0:
            return 1e10
        Q_calc = chezy_discharge(channel, y, S, C)
        return Q_calc - Q

    try:
        result = fsolve(equation, initial_guess, full_output=True)
        if result[2] == 1 and result[0][0] > 0:
            return result[0][0]
    except:
        pass

    try:
        y_n = brentq(equation, 0.001, max_depth)
        return y_n
    except:
        result = fsolve(equation, max_depth/2)
        return abs(result[0])


def critical_slope_chezy(channel, Q, C, y_c=None):
    """Calculate critical slope using Chézy equation (slope at which normal depth = critical depth)"""
    if y_c is None:
        y_c = critical_depth(channel, Q)

    A = channel.area(y_c)
    R = channel.hydraulic_radius(y_c)
    V = Q / A

    # From V = C * sqrt(R * S), solve for S
    S_c = (V / C)**2 / R
    return S_c


def manning_to_chezy(n, R):
    """Convert Manning's n to Chézy C: C = R^(1/6) / n"""
    return (R ** (1/6)) / n


def chezy_to_manning(C, R):
    """Convert Chézy C to Manning's n: n = R^(1/6) / C"""
    return (R ** (1/6)) / C


# ===== INVERSE CALCULATIONS =====

def estimate_manning_n(channel, Q, depth, S):
    """Estimate Manning's n from known discharge, depth, and slope

    Parameters:
    - channel: ChannelSection object
    - Q: discharge (m³/s)
    - depth: known flow depth (m)
    - S: channel slope (m/m)

    Returns:
    - n: Manning's roughness coefficient
    """
    A = channel.area(depth)
    R = channel.hydraulic_radius(depth)
    V = Q / A

    # From Manning's equation: V = (1/n) * R^(2/3) * S^(1/2)
    # Solve for n: n = R^(2/3) * S^(1/2) / V
    n = (R ** (2/3)) * (S ** 0.5) / V
    return n


def estimate_chezy_c(channel, Q, depth, S):
    """Estimate Chézy C from known discharge, depth, and slope

    Parameters:
    - channel: ChannelSection object
    - Q: discharge (m³/s)
    - depth: known flow depth (m)
    - S: channel slope (m/m)

    Returns:
    - C: Chézy coefficient (m^0.5/s)
    """
    A = channel.area(depth)
    R = channel.hydraulic_radius(depth)
    V = Q / A

    # From Chézy's equation: V = C * sqrt(R * S)
    # Solve for C: C = V / sqrt(R * S)
    C = V / math.sqrt(R * S)
    return C


print("Chézy equation and inverse calculation functions loaded successfully!")

Chézy equation and inverse calculation functions loaded successfully!


In [6]:
def manning_velocity(R, S, n):
    """Calculate velocity using Manning's equation: V = (1/n) * R^(2/3) * S^(1/2)"""
    return (1.0 / n) * (R ** (2/3)) * (S ** 0.5)


def manning_discharge(channel, depth, S, n):
    """Calculate discharge using Manning's equation"""
    A = channel.area(depth)
    R = channel.hydraulic_radius(depth)
    V = manning_velocity(R, S, n)
    return A * V


def normal_depth_manning(channel, Q, S, n, initial_guess=1.0, max_depth=20.0):
    """Calculate normal depth using Manning's equation"""
    def equation(y):
        if y <= 0:
            return 1e10
        Q_calc = manning_discharge(channel, y, S, n)
        return Q_calc - Q

    try:
        result = fsolve(equation, initial_guess, full_output=True)
        if result[2] == 1 and result[0][0] > 0:
            return result[0][0]
    except:
        pass

    try:
        y_n = brentq(equation, 0.001, max_depth)
        return y_n
    except:
        result = fsolve(equation, max_depth/2)
        return abs(result[0])


def critical_depth(channel, Q, initial_guess=1.0, max_depth=20.0):
    """Calculate critical depth where Froude number = 1"""
    def equation(y):
        if y <= 0:
            return 1e10
        A = channel.area(y)
        T = channel.top_width(y)
        if A <= 0 or T <= 0:
            return 1e10
        return Q**2 * T / (g * A**3) - 1.0

    try:
        y_c = brentq(equation, 0.001, max_depth)
        return y_c
    except:
        result = fsolve(equation, initial_guess)
        return abs(result[0])


def froude_number(channel, depth, Q):
    """Calculate Froude number: Fr = V / sqrt(g * D)"""
    A = channel.area(depth)
    if A == 0:
        return 0
    V = Q / A
    D = channel.hydraulic_depth(depth)
    if D == 0:
        return float('inf')
    return V / math.sqrt(g * D)


def flow_regime(Fr):
    """Determine flow regime from Froude number"""
    if Fr < 0.99:
        return 'subcritical'
    elif Fr > 1.01:
        return 'supercritical'
    else:
        return 'critical'


def specific_energy(channel, depth, Q):
    """Calculate specific energy: E = y + V²/(2g)"""
    A = channel.area(depth)
    if A == 0:
        return float('inf')
    V = Q / A
    return depth + V**2 / (2 * g)


def critical_slope_manning(channel, Q, n, y_c=None):
    """Calculate critical slope (slope at which normal depth = critical depth)"""
    if y_c is None:
        y_c = critical_depth(channel, Q)

    A = channel.area(y_c)
    R = channel.hydraulic_radius(y_c)
    V = Q / A

    S_c = (V * n / (R ** (2/3))) ** 2
    return S_c


def slope_classification(S, S_c):
    """Classify channel slope as mild, critical, or steep"""
    if S < S_c * 0.99:
        return 'mild'
    elif S > S_c * 1.01:
        return 'steep'
    else:
        return 'critical'


print("Flow calculation functions loaded successfully!")

Flow calculation functions loaded successfully!


In [7]:
# ===== IMPROVED CENTROID CALCULATIONS =====

def centroid_distance_circular_segment(R, depth):
    """Calculate distance from chord to centroid of circular segment

    Parameters:
    - R: radius of circle (m)
    - depth: depth of water from invert (m)

    Returns:
    - d_bar: distance from chord to centroid (m)

    Formula: d_bar = R * [2/3 * (sin³θ) / (θ - sin(θ)cos(θ))]
    where θ is the semi-angle subtended at center
    """
    if depth <= 0 or depth >= 2*R:
        return 0

    # Calculate semi-angle θ
    cos_theta = 1 - depth / R
    cos_theta = max(-1, min(1, cos_theta))
    theta = math.acos(cos_theta)

    sin_theta = math.sin(theta)

    # Calculate centroid distance
    numerator = 2 * (sin_theta ** 3) / 3
    denominator = theta - sin_theta * math.cos(theta)

    if abs(denominator) < 1e-10:
        return 0

    d_bar = R * numerator / denominator
    return d_bar


def centroid_height_triangular(depth, side_slope):
    """Calculate centroid height for triangular channel

    For V-shaped channel, centroid is at y_bar = y/3 from bottom
    """
    return depth / 3


def centroid_height(channel, depth):
    """Calculate centroid height (distance from bottom to centroid) for any channel

    Parameters:
    - channel: ChannelSection object
    - depth: flow depth (m)

    Returns:
    - y_bar: centroid height from bottom (m)
    """
    if isinstance(channel, RectangularChannel) or isinstance(channel, WideChannel):
        return depth / 2

    elif isinstance(channel, TriangularChannel):
        return depth / 3

    elif isinstance(channel, CircularChannel):
        # For circular channel, use centroid distance formula
        d_bar = centroid_distance_circular_segment(channel.radius, depth)
        # Centroid height from bottom = depth - d_bar
        return depth - d_bar

    elif isinstance(channel, TrapezoidalChannel):
        # For trapezoidal channel, calculate using moment of area
        b = channel.bottom_width
        m = channel.side_slope
        y = depth

        # Area = (b + my) * y
        # First moment = (b*y²/2) + (m*y³/3)
        A = (b + m * y) * y
        first_moment = (b * y**2 / 2) + (m * y**3 / 3)

        return first_moment / A if A > 0 else 0

    else:
        # Default: use hydraulic depth / 2 as approximation
        return channel.hydraulic_depth(depth) / 2


# ===== BAFFLE BLOCKS AND DRAG FORCES =====

def hydraulic_jump_with_baffle_blocks(channel, y1, Q, num_rows, block_height, width_basin, drag_coeff):
    """Calculate sequent depth for hydraulic jump with baffle blocks

    Parameters:
    - channel: ChannelSection object
    - y1: upstream depth (m)
    - Q: discharge (m³/s)
    - num_rows: number of rows of baffle blocks
    - block_height: effective height of blocks (m)
    - width_basin: width of stilling basin (m)
    - drag_coeff: drag coefficient c_D

    Returns:
    - y2: sequent depth downstream (m)
    - with_jump: boolean indicating if hydraulic jump occurs

    The drag force is calculated as: F_D = c_D * (1/2) * ρ * V² * A
    where A is the frontal area (blocks + gaps) = block_height * width_basin * num_rows
    """
    rho = 1000  # kg/m³

    # Calculate upstream conditions
    A1 = channel.area(y1)
    V1 = Q / A1
    y_bar1 = centroid_height(channel, y1)

    # Frontal area of baffle blocks
    A_frontal = block_height * width_basin  # per row

    # Total drag force from all rows
    F_drag_total = num_rows * drag_coeff * 0.5 * rho * V1**2 * A_frontal

    # Modified momentum equation with drag force
    # M1 - F_drag = M2
    # ρgAy_bar + ρQ²/A - F_drag = ρgAy_bar + ρQ²/A

    M1 = rho * g * A1 * y_bar1 + rho * Q**2 / A1
    M1_with_drag = M1 - F_drag_total

    # Solve for y2
    def momentum_equation(y2):
        if y2 <= 0:
            return 1e10
        A2 = channel.area(y2)
        if A2 == 0:
            return 1e10
        y_bar2 = centroid_height(channel, y2)
        M2 = rho * g * A2 * y_bar2 + rho * Q**2 / A2
        return M2 - M1_with_drag

    # Try to find sequent depth
    try:
        # First check if jump occurs (y2 > y1)
        y2_jump = brentq(momentum_equation, y1 * 1.01, y1 * 10)
        return y2_jump, True
    except:
        # No jump - solve for subcritical depth
        try:
            # Initial guess based on sequent depth without baffle blocks
            y2_no_blocks = sequent_depth(channel, y1, Q)
            y2_subcrit = fsolve(momentum_equation, y2_no_blocks)[0]
            return abs(y2_subcrit), False
        except:
            # Return original sequent depth as fallback
            return sequent_depth(channel, y1, Q), True


def force_on_baffle_blocks(channel, y1, y2, Q, num_rows, block_height, width_basin, drag_coeff):
    """Calculate total force on baffle blocks

    Returns the total force that the blocks must withstand
    """
    rho = 1000  # kg/m³

    A1 = channel.area(y1)
    V1 = Q / A1

    # Frontal area per row
    A_frontal = block_height * width_basin

    # Drag force
    F_drag = num_rows * drag_coeff * 0.5 * rho * V1**2 * A_frontal

    return F_drag


print("Improved centroid calculations and baffle blocks functions loaded successfully!")

Improved centroid calculations and baffle blocks functions loaded successfully!


In [8]:
def depth_from_specific_energy(channel, Q, E, regime='subcritical', initial_guess=None):
    """Find flow depth given specific energy and discharge

    Parameters:
    - channel: ChannelSection object
    - Q: discharge (m³/s)
    - E: specific energy (m)
    - regime: 'subcritical' or 'supercritical'
    - initial_guess: starting guess for solver (optional)

    Returns:
    - depth (m)
    """
    def equation(y):
        if y <= 0 or y > E:
            return 1e10
        E_calc = specific_energy(channel, y, Q)
        return E_calc - E

    # Get critical depth as reference
    y_c = critical_depth(channel, Q)

    if initial_guess is None:
        if regime == 'subcritical':
            initial_guess = max(y_c * 1.5, E * 0.8)
        else:  # supercritical
            initial_guess = y_c * 0.5

    try:
        if regime == 'subcritical':
            # Subcritical depth is > y_c
            y = brentq(equation, y_c * 1.001, E * 0.999)
        else:  # supercritical
            # Supercritical depth is < y_c
            y = brentq(equation, 0.001, y_c * 0.999)
        return y
    except:
        result = fsolve(equation, initial_guess)
        return abs(result[0])


def alternate_depth(channel, Q, y_known):
    """Find the alternate depth (same specific energy, different regime)

    Parameters:
    - channel: ChannelSection object
    - Q: discharge (m³/s)
    - y_known: known depth (m)

    Returns:
    - y_alternate: alternate depth (m)
    """
    E = specific_energy(channel, y_known, Q)
    y_c = critical_depth(channel, Q)

    Fr = froude_number(channel, y_known, Q)

    if Fr < 1:
        # Known depth is subcritical, find supercritical alternate
        return depth_from_specific_energy(channel, Q, E, 'supercritical')
    else:
        # Known depth is supercritical, find subcritical alternate
        return depth_from_specific_energy(channel, Q, E, 'subcritical')


print("Depth from specific energy functions loaded!")


Depth from specific energy functions loaded!


## 5. Hydraulic Jump Analysis

Functions for analyzing hydraulic jumps, sequent depths, and energy losses.

In [9]:
# ===== GVF WITH CHÉZY EQUATION =====

def gvf_derivative_chezy(channel, y, Q, S0, C):
    """Calculate dy/dx for GVF using Chézy equation"""
    if y <= 0:
        return 0

    A = channel.area(y)
    R = channel.hydraulic_radius(y)
    T = channel.top_width(y)

    if A == 0 or T == 0:
        return 0

    V = Q / A

    # Friction slope using Chézy: V = C * sqrt(R * S_f)
    # So: S_f = (V / C)² / R
    Sf = (V / C)**2 / R

    # Froude number
    D = channel.hydraulic_depth(y)
    if D <= 0:
        Fr = float('inf')
    else:
        Fr = V / math.sqrt(g * D)

    # GVF equation: dy/dx = (S0 - Sf) / (1 - Fr²)
    numerator = S0 - Sf
    denominator = 1 - Fr**2

    if abs(denominator) < 1e-10:
        return 0

    return numerator / denominator


def solve_gvf_profile_chezy(channel, Q, S0, C, y_start, distance, num_steps, direction='downstream'):
    """Solve water surface profile using Euler method with Chézy equation"""
    dx = distance / num_steps

    if direction == 'upstream':
        dx = -dx

    x = np.zeros(num_steps + 1)
    y = np.zeros(num_steps + 1)

    x[0] = 0
    y[0] = y_start

    for i in range(num_steps):
        dy_dx = gvf_derivative_chezy(channel, y[i], Q, S0, C)
        y[i+1] = y[i] + dy_dx * dx
        x[i+1] = x[i] + dx

        if y[i+1] < 0.001:
            y[i+1] = 0.001

    return x, y


def distance_to_depth_chezy(channel, Q, S0, C, y_start, y_target, num_steps=100, max_distance=10000):
    """Calculate distance from y_start to y_target using GVF with Chézy equation"""
    if abs(y_start - y_target) < 0.001:
        return 0

    direction = 'upstream' if y_target > y_start else 'downstream'
    distance = abs(y_target - y_start) * 100

    for iteration in range(10):
        x, y = solve_gvf_profile_chezy(channel, Q, S0, C, y_start, distance, num_steps, direction)
        y_end = y[-1]

        if abs(y_end - y_target) < 0.01:
            return abs(x[-1])

        if direction == 'upstream':
            distance = distance * 1.5 if y_end < y_target else distance * 0.7
        else:
            distance = distance * 1.5 if y_end > y_target else distance * 0.7

        if distance > max_distance:
            distance = max_distance
            break

    return abs(x[-1])


print("GVF functions with Chézy equation loaded successfully!")

GVF functions with Chézy equation loaded successfully!


In [10]:
def sequent_depth_rectangular(y1, Q, width):
    """Calculate sequent depth for hydraulic jump in rectangular channel"""
    A1 = width * y1
    V1 = Q / A1
    Fr1 = V1 / math.sqrt(g * y1)

    # Analytical solution for rectangular channel
    y2 = (y1 / 2) * (-1 + math.sqrt(1 + 8 * Fr1**2))
    return y2


def sequent_depth_general(channel, y1, Q, initial_guess=None):
    """Calculate sequent depth using momentum equation for general channel shapes"""
    def momentum(y):
        if y <= 0:
            return 0
        A = channel.area(y)
        if A == 0:
            return 0
        # y_bar = channel.hydraulic_depth(y) / 2
        y_bar = centroid_height(channel, y)
        M = Q**2 / (g * A) + A * y_bar
        return M

    M1 = momentum(y1)

    if initial_guess is None:
        A1 = channel.area(y1)
        T1 = channel.top_width(y1)
        if T1 > 0:
            equiv_y1 = A1 / T1
            initial_guess = sequent_depth_rectangular(equiv_y1, Q, T1)
        else:
            initial_guess = y1 * 2

    def equation(y2):
        if y2 <= 0:
            return 1e10
        return momentum(y2) - M1

    try:
        result = fsolve(equation, initial_guess, full_output=True)
        if result[2] == 1:
            return abs(result[0][0])
    except:
        pass

    try:
        if initial_guess > y1:
            y2 = brentq(equation, y1 * 1.01, y1 * 10)
        else:
            y2 = brentq(equation, 0.001, y1 * 0.99)
        return y2
    except:
        result = fsolve(equation, initial_guess)
        return abs(result[0])


def sequent_depth(channel, y1, Q, initial_guess=None):
    """Calculate sequent depth (automatically selects appropriate method)"""
    if isinstance(channel, RectangularChannel):
        return sequent_depth_rectangular(y1, Q, channel.width)
    else:
        return sequent_depth_general(channel, y1, Q, initial_guess)


def energy_loss_jump(channel, y1, y2, Q):
    """Calculate energy loss in hydraulic jump: ΔE = E1 - E2"""
    A1 = channel.area(y1)
    A2 = channel.area(y2)

    V1 = Q / A1
    V2 = Q / A2

    E1 = y1 + V1**2 / (2 * g)
    E2 = y2 + V2**2 / (2 * g)

    return E1 - E2


def energy_loss_fraction(channel, y1, y2, Q):
    """Calculate fraction of energy dissipated: f = ΔE / E1"""
    A1 = channel.area(y1)
    V1 = Q / A1
    E1 = y1 + V1**2 / (2 * g)

    delta_E = energy_loss_jump(channel, y1, y2, Q)

    return delta_E / E1


def force_on_obstacle(channel, y1, y2, Q):
    """Calculate force on obstacle causing hydraulic jump"""
    rho = 1000  # Water density (kg/m³)

    A1 = channel.area(y1)
    A2 = channel.area(y2)

    V1 = Q / A1
    V2 = Q / A2

    momentum_change = rho * Q * (V1 - V2)

    if isinstance(channel, RectangularChannel):
          pressure_change = rho * g * (A1 * y1/2 - A2 * y2/2)
    else:
          y_bar1 = centroid_height(channel, y1)
          y_bar2 = centroid_height(channel, y2)
          pressure_change = rho * g * (A1 * y_bar1 - A2 * y_bar2)

    return momentum_change + pressure_change


print("Hydraulic jump functions loaded successfully!")

Hydraulic jump functions loaded successfully!


## 6. Gradually Varied Flow (GVF)

Functions for solving water surface profiles and GVF problems.

In [11]:
def gvf_derivative_manning(channel, y, Q, S0, n):
    """Calculate dy/dx for GVF using Manning's equation"""
    if y <= 0:
        return 0

    A = channel.area(y)
    R = channel.hydraulic_radius(y)
    T = channel.top_width(y)

    if A == 0 or T == 0:
        return 0

    V = Q / A

    # Friction slope
    Sf = (n * V / (R ** (2/3))) ** 2

    # Froude number
    D = channel.hydraulic_depth(y)
    if D <= 0:
        Fr = float('inf')
    else:
        Fr = V / math.sqrt(g * D)

    # GVF equation: dy/dx = (S0 - Sf) / (1 - Fr²)
    numerator = S0 - Sf
    denominator = 1 - Fr**2

    if abs(denominator) < 1e-10:
        return 0

    return numerator / denominator


def solve_gvf_profile_manning(channel, Q, S0, n, y_start, distance, num_steps, direction='downstream'):
    """Solve water surface profile using Euler method"""
    dx = distance / num_steps

    if direction == 'upstream':
        dx = -dx

    x = np.zeros(num_steps + 1)
    y = np.zeros(num_steps + 1)

    x[0] = 0
    y[0] = y_start

    for i in range(num_steps):
        dy_dx = gvf_derivative_manning(channel, y[i], Q, S0, n)
        y[i+1] = y[i] + dy_dx * dx
        x[i+1] = x[i] + dx

        if y[i+1] < 0.001:
            y[i+1] = 0.001

    return x, y


def distance_to_depth_manning(channel, Q, S0, n, y_start, y_target, num_steps=100, max_distance=10000):
    """Calculate distance from y_start to y_target using GVF"""
    if abs(y_start - y_target) < 0.001:
        return 0

    direction = 'upstream' if y_target > y_start else 'downstream'
    distance = abs(y_target - y_start) * 100

    for iteration in range(10):
        x, y = solve_gvf_profile_manning(channel, Q, S0, n, y_start, distance, num_steps, direction)
        y_end = y[-1]

        if abs(y_end - y_target) < 0.01:
            return abs(x[-1])

        if direction == 'upstream':
            distance = distance * 1.5 if y_end < y_target else distance * 0.7
        else:
            distance = distance * 1.5 if y_end > y_target else distance * 0.7

        if distance > max_distance:
            distance = max_distance
            break

    return abs(x[-1])


def classify_gvf_curve(S0, S_c, y, y_n, y_c):
    """Classify GVF curve type (M1, M2, S1, S2, etc.)"""
    # Slope type
    if abs(S0) < 1e-10:
        slope_type = 'H'
    elif S0 < 0:
        slope_type = 'A'
    elif abs(S0 - S_c) < S_c * 0.01:
        slope_type = 'C'
    elif S0 < S_c:
        slope_type = 'M'
    else:
        slope_type = 'S'

    # Zone determination
    if slope_type == 'M':
        if y > y_n:
            zone = '1'
        elif y > y_c:
            zone = '2'
        else:
            zone = '3'
    elif slope_type == 'S':
        if y > y_c:
            zone = '1'
        elif y > y_n:
            zone = '2'
        else:
            zone = '3'
    elif slope_type == 'C':
        zone = '1' if y > y_c else '3'
    elif slope_type in ['H', 'A']:
        zone = '2' if y > y_c else '3'
    else:
        zone = '?'

    return slope_type + zone


print("GVF functions loaded successfully!")

GVF functions loaded successfully!


## 7. Weir and Sluice Gate Analysis

Functions for analyzing flow over weirs and under sluice gates.

In [12]:
def weir_upstream_depth(channel, Q, weir_height, normal_depth=None):
    """Calculate upstream depth for flow over a broad-crested weir"""
    y_c = critical_depth(channel, Q)
    E_c = (3/2) * y_c
    E_upstream = E_c + weir_height

    def equation(y):
        if y <= weir_height:
            return 1e10
        E = specific_energy(channel, y, Q)
        return E - E_upstream

    try:
        y_upstream = brentq(equation, weir_height + y_c, E_upstream * 2)
        return y_upstream
    except:
        initial_guess = weir_height + y_c * 1.5
        result = fsolve(equation, initial_guess)
        return abs(result[0])


def weir_depth_over_crest(channel, Q, weir_height, E_upstream):
    """Calculate depth over the crest of a broad-crested weir"""
    E_crest = E_upstream - weir_height
    y_c = critical_depth(channel, Q)

    def equation(y):
        if y <= 0:
            return 1e10
        E = specific_energy(channel, y, Q)
        return E - E_crest

    try:
        y_over_crest = brentq(equation, 0.001, E_crest)
        return y_over_crest
    except:
        result = fsolve(equation, y_c)
        return abs(result[0])


def sluice_gate_discharge(channel, y_upstream, y_downstream, energy_loss_coeff=0.0):
    """Calculate discharge under a sluice gate using Bernoulli equation"""
    A_down = channel.area(y_downstream)

    if y_upstream <= y_downstream:
        return 0

    V2 = math.sqrt(2 * g * (y_upstream - y_downstream) / (1 + energy_loss_coeff))
    Q = A_down * V2

    return Q


def sluice_gate_upstream_depth(channel, Q, y_downstream):
    """Calculate upstream depth for given discharge and downstream depth"""
    E2 = specific_energy(channel, y_downstream, Q)

    def equation(y):
        if y <= y_downstream:
            return 1e10
        E1 = specific_energy(channel, y, Q)
        return E1 - E2

    try:
        y_c = critical_depth(channel, Q)
        y_upstream = brentq(equation, max(y_c, y_downstream), E2 * 2)
        return y_upstream
    except:
        initial_guess = y_downstream * 3
        result = fsolve(equation, initial_guess)
        return abs(result[0])


def sluice_gate_force(channel, y_upstream, y_downstream, Q):
    """Calculate force on sluice gate using momentum equation"""
    rho = 1000  # Water density (kg/m³)

    A1 = channel.area(y_upstream)
    A2 = channel.area(y_downstream)

    V1 = Q / A1 if A1 > 0 else 0
    V2 = Q / A2 if A2 > 0 else 0

    momentum_change = rho * Q * (V1 - V2)

    D1 = channel.hydraulic_depth(y_upstream)
    D2 = channel.hydraulic_depth(y_downstream)

    pressure_change = rho * g * (A1 * D1/2 - A2 * D2/2)

    return momentum_change + pressure_change


def free_overfall_depth(channel, Q):
    """Calculate depth at a free overfall (approximately 71.5% of critical depth)"""
    y_c = critical_depth(channel, Q)
    return 0.715 * y_c


print("Weir and sluice gate functions loaded successfully!")

Weir and sluice gate functions loaded successfully!


## 8. High-Level Solver Class

A comprehensive solver class that provides a unified interface for all types of open channel flow problems.

In [13]:
class OpenChannelSolver:
    """Main solver class for open channel flow problems"""

    def __init__(self):
        self.g = 9.81

    def solve_basic_flow_problem(self, channel_type, channel_params, Q, S, n=None, C=None):
        """Solve basic flow problem: normal depth, critical depth, Froude number, etc."""
        channel = self._create_channel(channel_type, channel_params)

        results = {
            'channel_type': channel_type,
            'discharge': Q
        }

        # Critical depth
        y_c = critical_depth(channel, Q)
        results['critical_depth'] = y_c

        # Process slopes
        if not isinstance(S, list):
            S = [S]

        for i, slope in enumerate(S):
            slope_results = {}

            if n is not None:
                y_n = normal_depth_manning(channel, Q, slope, n)
                slope_results['normal_depth'] = y_n

                Fr_n = froude_number(channel, y_n, Q)
                slope_results['froude_number'] = Fr_n
                slope_results['flow_regime'] = flow_regime(Fr_n)

            results[f'slope_{i+1}'] = slope_results

        # Critical slope
        if n is not None:
            S_c = critical_slope_manning(channel, Q, n, y_c)
            results['critical_slope'] = S_c

        return results

    def solve_hydraulic_jump_problem(self, channel_type, channel_params, Q, y1=None, y2=None):
        """Solve hydraulic jump problems"""
        channel = self._create_channel(channel_type, channel_params)

        results = {'discharge': Q}

        if y1 is not None:
            results['depth_1'] = y1
            Fr1 = froude_number(channel, y1, Q)
            results['froude_1'] = Fr1

            y2_calc = sequent_depth(channel, y1, Q)
            results['sequent_depth'] = y2_calc

            dE = energy_loss_jump(channel, y1, y2_calc, Q)
            frac = energy_loss_fraction(channel, y1, y2_calc, Q)
            results['energy_loss'] = dE
            results['energy_loss_fraction'] = frac

        return results

    def solve_gvf_problem(self, channel_type, channel_params, Q, S, n, y_start, y_target, num_steps=100):
        """Solve gradually varied flow problem"""
        channel = self._create_channel(channel_type, channel_params)

        results = {
            'discharge': Q,
            'slope': S,
            'manning_n': n,
            'start_depth': y_start,
            'target_depth': y_target
        }

        y_n = normal_depth_manning(channel, Q, S, n)
        y_c = critical_depth(channel, Q)
        S_c = critical_slope_manning(channel, Q, n, y_c)

        results['normal_depth'] = y_n
        results['critical_depth'] = y_c
        results['critical_slope'] = S_c
        results['slope_classification'] = slope_classification(S, S_c)

        gvf_curve = classify_gvf_curve(S, S_c, y_start, y_n, y_c)
        results['gvf_curve'] = gvf_curve

        distance = distance_to_depth_manning(channel, Q, S, n, y_start, y_target, num_steps)
        results['distance'] = distance

        return results

    def _create_channel(self, channel_type, params):
        """Factory method to create channel object"""
        if channel_type == 'rectangular':
            return RectangularChannel(params['width'])
        elif channel_type == 'trapezoidal':
            return TrapezoidalChannel(params['bottom_width'], params['side_slope'])
        elif channel_type == 'circular':
            return CircularChannel(params['diameter'])
        elif channel_type == 'triangular':
            if 'side_slope' in params:
                return TriangularChannel(side_slope=params['side_slope'])
            else:
                return TriangularChannel(semi_angle=params['semi_angle'])
        elif channel_type == 'wide':
            return WideChannel()
        else:
            raise ValueError(f"Unknown channel type: {channel_type}")

    def print_results(self, results, title="Results"):
        """Pretty print results"""
        print("\n" + "="*70)
        print(title)
        print("="*70)

        for key, value in results.items():
            if isinstance(value, dict):
                print(f"\n{key}:")
                for k, v in value.items():
                    if isinstance(v, (int, float)):
                        print(f"  {k}: {v:.4f}")
                    else:
                        print(f"  {k}: {v}")
            elif isinstance(value, (int, float)):
                print(f"{key}: {value:.4f}")
            else:
                print(f"{key}: {value}")


print("OpenChannelSolver class loaded successfully!")
print("\n" + "="*70)
print("ALL MODULES LOADED - READY TO SOLVE PROBLEMS!")
print("="*70)

OpenChannelSolver class loaded successfully!

ALL MODULES LOADED - READY TO SOLVE PROBLEMS!


## 9. Example Problems

Here are several worked examples demonstrating how to use the solver.

### Example 1: Rectangular Channel - Normal and Critical Depths

In [14]:
# Create solver
solver = OpenChannelSolver()

# Solve for rectangular channel with width = 5m, Q = 20 m³/s, n = 0.02
results = solver.solve_basic_flow_problem(
    channel_type='rectangular',
    channel_params={'width': 5.0},
    Q=20.0,
    S=[0.001, 0.01],  # Two different slopes
    n=0.02
)

solver.print_results(results, "Example 1: Rectangular Channel")


Example 1: Rectangular Channel
channel_type: rectangular
discharge: 20.0000
critical_depth: 1.1771

slope_1:
  normal_depth: 2.2575
  froude_number: 0.3765
  flow_regime: subcritical

slope_2:
  normal_depth: 1.0008
  froude_number: 1.2756
  flow_regime: supercritical
critical_slope: 0.0062


### Example 2: Trapezoidal Channel

In [15]:
# Trapezoidal channel: bottom width = 0.6m, side slope = 0.75, Q = 2.6 m³/s
results = solver.solve_basic_flow_problem(
    channel_type='trapezoidal',
    channel_params={'bottom_width': 0.6, 'side_slope': 0.75},
    Q=2.6,
    S=1/2500,
    n=0.012
)

solver.print_results(results, "Example 2: Trapezoidal Channel")


Example 2: Trapezoidal Channel
channel_type: trapezoidal
discharge: 2.6000
critical_depth: 0.8733

slope_1:
  normal_depth: 1.3935
  froude_number: 0.3923
  flow_regime: subcritical
critical_slope: 0.0028


### Example 3: Circular Channel (Pipe)

In [16]:
# Circular channel: diameter = 1.4m (radius = 0.7m), Q = 0.8 m³/s
results = solver.solve_basic_flow_problem(
    channel_type='circular',
    channel_params={'diameter': 1.4},
    Q=0.8,
    S=0.02,  # 2% slope
    n=0.013
)

solver.print_results(results, "Example 3: Circular Channel")


Example 3: Circular Channel
channel_type: circular
discharge: 0.8000
critical_depth: 0.4605

slope_1:
  normal_depth: 0.2933
  froude_number: 2.4055
  flow_regime: supercritical
critical_slope: 0.0034


/tmp/ipython-input-3869205438.py:84: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return math.acos(cos_theta)


### Example 4: Hydraulic Jump in Rectangular Channel

In [17]:
# Rectangular channel: width = 3m, upstream depth = 0.3m after sluice gate
channel = RectangularChannel(width=3.0)
Q = 13.0  # m³/s
y1 = 0.3  # m

# Calculate sequent depth
y2 = sequent_depth(channel, y1, Q)
print(f"Upstream depth (supercritical): y1 = {y1:.3f} m")
print(f"Sequent depth (subcritical): y2 = {y2:.3f} m")

# Calculate energy loss
dE = energy_loss_jump(channel, y1, y2, Q)
frac = energy_loss_fraction(channel, y1, y2, Q)
print(f"\nEnergy loss in jump: ΔE = {dE:.3f} m")
print(f"Energy loss fraction: {frac*100:.1f}%")

# Froude numbers
Fr1 = froude_number(channel, y1, Q)
Fr2 = froude_number(channel, y2, Q)
print(f"\nFroude number upstream: Fr1 = {Fr1:.3f} ({flow_regime(Fr1)})")
print(f"Froude number downstream: Fr2 = {Fr2:.3f} ({flow_regime(Fr2)})")

Upstream depth (supercritical): y1 = 0.300 m
Sequent depth (subcritical): y2 = 3.425 m

Energy loss in jump: ΔE = 7.427 m
Energy loss fraction: 67.9%

Froude number upstream: Fr1 = 8.420 (supercritical)
Froude number downstream: Fr2 = 0.218 (subcritical)


### Example 5: Sluice Gate Problem

In [18]:
# Sluice gate in rectangular channel
channel = RectangularChannel(width=3.0)
y_upstream = 2.0  # m
y_downstream_gate = 0.3  # m (depth just after gate)

# Calculate discharge
Q = sluice_gate_discharge(channel, y_upstream, y_downstream_gate)
print(f"Upstream depth: {y_upstream} m")
print(f"Downstream depth (at gate): {y_downstream_gate} m")
print(f"Discharge: Q = {Q:.3f} m³/s")

# Normal depth downstream (given slope and n)
S = 1/1000
n = 0.014
y_n = normal_depth_manning(channel, Q, S, n)
print(f"\nNormal depth downstream: y_n = {y_n:.3f} m")

# Force on gate
F = sluice_gate_force(channel, y_upstream, y_downstream_gate, Q)
print(f"Force on gate: F = {F/1000:.1f} kN")

Upstream depth: 2.0 m
Downstream depth (at gate): 0.3 m
Discharge: Q = 5.198 m³/s

Normal depth downstream: y_n = 1.055 m
Force on gate: F = 32.0 kN


### Example 6: Gradually Varied Flow (GVF) - Free Overfall

In [19]:
# Wide channel with free overfall
results = solver.solve_gvf_problem(
    channel_type='wide',
    channel_params={},
    Q=0.5,  # m³/s per m width
    S=2e-5,
    n=0.01,
    y_start=0.715 * critical_depth(WideChannel(), 0.5),  # Depth at overfall
    y_target=1.0,  # Target depth upstream
    num_steps=100
)

solver.print_results(results, "Example 6: GVF - Free Overfall")


Example 6: GVF - Free Overfall
discharge: 0.5000
slope: 0.0000
manning_n: 0.0100
start_depth: 0.2104
target_depth: 1.0000
normal_depth: 1.0692
critical_depth: 0.2943
critical_slope: 0.0015
slope_classification: mild
gvf_curve: M3
distance: 3035.4554


### Example 7: Broad-Crested Weir

In [20]:
# Wide channel with broad-crested weir
channel = WideChannel()
Q = 1.5  # m³/s per m width
S = 2e-4
n = 0.015
weir_height = 0.2  # m

# Normal and critical depths
y_n = normal_depth_manning(channel, Q, S, n)
y_c = critical_depth(channel, Q)

print(f"Normal depth: y_n = {y_n:.3f} m")
print(f"Critical depth: y_c = {y_c:.3f} m")

# Upstream depth for critical flow over weir
y_upstream = weir_upstream_depth(channel, Q, weir_height)
print(f"\nWeir height: {weir_height} m")
print(f"Upstream depth: {y_upstream:.3f} m")
print(f"Depth over weir crest: {y_c:.3f} m (critical depth)")
print(f"Downstream depth: {y_n:.3f} m (returns to normal)")

Normal depth: y_n = 1.321 m
Critical depth: y_c = 0.612 m

Weir height: 0.2 m
Upstream depth: 1.005 m
Depth over weir crest: 0.612 m (critical depth)
Downstream depth: 1.321 m (returns to normal)


### Example 8: Triangular Channel with Hydraulic Jump

In [21]:
# Triangular channel with semi-angle = 40°
results = solver.solve_hydraulic_jump_problem(
    channel_type='triangular',
    channel_params={'semi_angle': 40},
    Q=16.0,
    y1=1.85
)

solver.print_results(results, "Example 8: Triangular Channel - Hydraulic Jump")


Example 8: Triangular Channel - Hydraulic Jump
discharge: 16.0000
depth_1: 1.8500
froude_1: 1.8495
sequent_depth: 1.8500
energy_loss: 0.0000
energy_loss_fraction: 0.0000


### Example 9: Compound Channel (River with Floodplains)

In [22]:
# Compound channel: River with main channel and floodplains
# Main channel: rectangular, 10m wide, depth changes at 2m
# Floodplains: trapezoidal extension with side slope 5:1

channel = CompoundChannel(
    bottom_section_type='rectangular',
    bottom_params={'width': 10.0},
    break_depth=2.0,
    top_section_type='trapezoidal',
    top_params={'bottom_width': 10.0, 'side_slope': 5.0}
)

# High flow discharge
Q = 150.0  # m³/s
S = 0.0005  # slope
n = 0.025  # Manning's n (higher due to vegetated floodplains)

print("COMPOUND CHANNEL ANALYSIS")
print("="*70)
print("\nChannel Configuration:")
print("  Main channel: Rectangular, 10m wide")
print("  Break depth: 2.0 m (bankfull depth)")
print("  Floodplains: Trapezoidal, 5H:1V side slopes")
print("\nFlow Conditions:")
print(f"  Discharge: Q = {Q} m³/s")
print(f"  Slope: S = {S}")
print(f"  Manning's n = {n}")

# Calculate normal depth
y_n = normal_depth_manning(channel, Q, S, n)
print(f"\nNormal depth: y_n = {y_n:.3f} m")

# Check if flow is in main channel only or spills to floodplains
if y_n <= channel.break_depth:
    print("  → Flow confined to main channel")
else:
    print(f"  → Flow depth {y_n - channel.break_depth:.3f} m above bankfull")
    print("  → Floodplains are inundated")

# Calculate areas and widths at normal depth
A_n = channel.area(y_n)
T_n = channel.top_width(y_n)
R_n = channel.hydraulic_radius(y_n)

print(f"\nGeometric Properties at Normal Depth:")
print(f"  Flow area: A = {A_n:.2f} m²")
print(f"  Top width: T = {T_n:.2f} m")
print(f"  Hydraulic radius: R = {R_n:.3f} m")

# Compare with flow at break depth (bankfull)
A_break = channel.area(channel.break_depth)
T_break = channel.top_width(channel.break_depth)
print(f"\nGeometric Properties at Bankfull (y = {channel.break_depth} m):")
print(f"  Flow area: A = {A_break:.2f} m²")
print(f"  Top width: T = {T_break:.2f} m")

# Calculate critical depth
y_c = critical_depth(channel, Q)
print(f"\nCritical depth: y_c = {y_c:.3f} m")

# Froude number
Fr = froude_number(channel, y_n, Q)
print(f"Froude number: Fr = {Fr:.3f}")
print(f"Flow regime: {flow_regime(Fr)}")

# Velocity
V_n = Q / A_n
print(f"\nMean velocity: V = {V_n:.3f} m/s")

print("\n" + "="*70)

COMPOUND CHANNEL ANALYSIS

Channel Configuration:
  Main channel: Rectangular, 10m wide
  Break depth: 2.0 m (bankfull depth)
  Floodplains: Trapezoidal, 5H:1V side slopes

Flow Conditions:
  Discharge: Q = 150.0 m³/s
  Slope: S = 0.0005
  Manning's n = 0.025

Normal depth: y_n = 5.118 m
  → Flow depth 3.118 m above bankfull
  → Floodplains are inundated

Geometric Properties at Normal Depth:
  Flow area: A = 99.78 m²
  Top width: T = 41.18 m
  Hydraulic radius: R = 2.179 m

Geometric Properties at Bankfull (y = 2.0 m):
  Flow area: A = 20.00 m²
  Top width: T = 10.00 m

Critical depth: y_c = 3.055 m
Froude number: Fr = 0.308
Flow regime: subcritical

Mean velocity: V = 1.503 m/s



## 11. WEB INTERFACE SETUP

Now let's create a web interface for easy calculations!

### Step 1: Install Flask and ngrok

In [23]:
# Install Flask and pyngrok for web interface
!pip install flask pyngrok -q

print("Flask and ngrok installed successfully!")

Flask and ngrok installed successfully!


### Step 2: Create Flask Web Application

In [24]:
from flask import Flask, request, render_template_string
import sys
from io import StringIO

app = Flask(__name__)

# HTML Template for the main page
HOME_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Open Channel Flow Calculator</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }

        .container {
            max-width: 900px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            padding: 40px;
        }

        h1 {
            color: #667eea;
            text-align: center;
            margin-bottom: 10px;
            font-size: 2.5em;
        }

        .subtitle {
            text-align: center;
            color: #666;
            margin-bottom: 40px;
            font-size: 1.1em;
        }

        .calc-type-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }

        .calc-card {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            cursor: pointer;
            transition: transform 0.3s, box-shadow 0.3s;
            text-align: center;
        }

        .calc-card:hover {
            transform: translateY(-5px);
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        }

        .calc-card h3 {
            margin-bottom: 10px;
            font-size: 1.3em;
        }

        .calc-card p {
            font-size: 0.9em;
            opacity: 0.9;
        }

        .form-section {
            display: none;
            animation: fadeIn 0.5s;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .form-section.active {
            display: block;
        }

        .form-group {
            margin-bottom: 20px;
        }

        label {
            display: block;
            margin-bottom: 8px;
            color: #333;
            font-weight: 600;
        }

        input, select {
            width: 100%;
            padding: 12px;
            border: 2px solid #e0e0e0;
            border-radius: 8px;
            font-size: 16px;
            transition: border-color 0.3s;
        }

        input:focus, select:focus {
            outline: none;
            border-color: #667eea;
        }

        button {
            width: 100%;
            padding: 15px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 8px;
            font-size: 18px;
            font-weight: 600;
            cursor: pointer;
            transition: transform 0.2s;
        }

        button:hover {
            transform: scale(1.02);
        }

        .back-btn {
            background: #666;
            margin-bottom: 20px;
        }

        .info-text {
            background: #f0f4ff;
            padding: 15px;
            border-radius: 8px;
            margin-bottom: 20px;
            color: #666;
            font-size: 0.9em;
        }

        .channel-params {
            background: #f9f9f9;
            padding: 20px;
            border-radius: 8px;
            margin-bottom: 20px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🌊 Open Channel Flow Calculator</h1>
        <p class="subtitle">Professional Hydraulic Engineering Tool</p>

        <div id="calcSelection">
            <h2 style="margin-bottom: 20px; color: #333;">Select Calculation Type:</h2>
            <div class="calc-type-grid">
                <div class="calc-card" onclick="showForm('normal')">
                    <h3>📏 Normal & Critical Depth</h3>
                    <p>Calculate flow depths and Froude number</p>
                </div>
                <div class="calc-card" onclick="showForm('jump')">
                    <h3>⚡ Hydraulic Jump</h3>
                    <p>Sequent depth and energy loss</p>
                </div>
                <div class="calc-card" onclick="showForm('weir')">
                    <h3>🚧 Weir Analysis</h3>
                    <p>Flow over broad-crested weirs</p>
                </div>
                <div class="calc-card" onclick="showForm('gate')">
                    <h3>🚪 Sluice Gate</h3>
                    <p>Discharge and gate forces</p>
                </div>
                 <div class="calc-card" onclick="showForm('gvf')">
                    <h3>📈 Gradually Varied Flow</h3>
                    <p>Water surface profiles and distances</p>
                </div>
            </div>
        </div>

        <!-- Normal Depth Form -->
        <div id="normalForm" class="form-section">
            <button class="back-btn" onclick="showSelection()">← Back to Selection</button>
            <h2 style="margin-bottom: 20px; color: #333;">Normal & Critical Depth Calculation</h2>
            <div class="info-text">
                Calculate normal depth, critical depth, Froude number, and flow regime for uniform flow conditions.
            </div>
            <form action="/calculate/normal" method="post">
                <div class="form-group">
                    <label>Channel Type:</label>
                    <select name="channel_type" onchange="updateChannelParams(this.value)" required>
                        <option value="rectangular">Rectangular</option>
                        <option value="trapezoidal">Trapezoidal</option>
                        <option value="circular">Circular (Pipe)</option>
                        <option value="triangular">Triangular</option>
                        <option value="wide">Wide Channel</option>
                        <option value="compound">Compound (River with Floodplains)</option>
                    </select>
                </div>
                <div id="channelParams" class="channel-params">
                    <div class="form-group">
                        <label>Width (m):</label>
                        <input type="number" step="any" name="width" value="5.0" required>
                    </div>
                </div>
                <div class="form-group">
                    <label>Discharge Q (m³/s):</label>
                    <input type="number" step="any" name="Q" value="20.0" required>
                </div>
                <div class="form-group">
                    <label>Slope S (m/m):</label>
                    <input type="number" step="any" name="S" value="0.001" required>
                </div>
                <div class="form-group">
                    <label>Manning's n:</label>
                    <input type="number" step="any" name="n" value="0.015" required>
                </div>
                <button type="submit">Calculate</button>
            </form>
        </div>

        <!-- Hydraulic Jump Form -->
        <div id="jumpForm" class="form-section">
            <button class="back-btn" onclick="showSelection()">← Back to Selection</button>
            <h2 style="margin-bottom: 20px; color: #333;">Hydraulic Jump Analysis</h2>
            <div class="info-text">
                Calculate sequent depth, energy loss, and other properties of a hydraulic jump.
            </div>
            <form action="/calculate/jump" method="post">
                <div class="form-group">
                    <label>Channel Type:</label>
                    <select name="channel_type" onchange="updateJumpParams(this.value)" required>
                        <option value="rectangular">Rectangular</option>
                        <option value="trapezoidal">Trapezoidal</option>
                        <option value="triangular">Triangular</option>
                    </select>
                </div>
                <div id="jumpChannelParams" class="channel-params">
                    <div class="form-group">
                        <label>Width (m):</label>
                        <input type="number" step="any" name="width" value="3.0" required>
                    </div>
                </div>
                <div class="form-group">
                    <label>Discharge Q (m³/s):</label>
                    <input type="number" step="any" name="Q" value="13.0" required>
                </div>
                <div class="form-group">
                    <label>Upstream Depth y₁ (m):</label>
                    <input type="number" step="any" name="y1" value="0.3" required>
                </div>
                <button type="submit">Calculate</button>
            </form>
        </div>

        <!-- Weir Form -->
        <div id="weirForm" class="form-section">
            <button class="back-btn" onclick="showSelection()">← Back to Selection</button>
            <h2 style="margin-bottom: 20px; color: #333;">Broad-Crested Weir Analysis</h2>
            <div class="info-text">
                Calculate upstream depth for flow over a broad-crested weir.
            </div>
            <form action="/calculate/weir" method="post">
                <div class="form-group">
                    <label>Channel Type:</label>
                    <select name="channel_type" onchange="updateWeirParams(this.value)" required>
                        <option value="rectangular">Rectangular</option>
                        <option value="wide">Wide Channel</option>
                    </select>
                </div>
                <div id="weirChannelParams" class="channel-params">
                    <div class="form-group">
                        <label>Width (m):</label>
                        <input type="number" step="any" name="width" value="3.0" required>
                    </div>
                </div>
                <div class="form-group">
                    <label>Discharge Q (m³/s):</label>
                    <input type="number" step="any" name="Q" value="1.5" required>
                </div>
                <div class="form-group">
                    <label>Weir Height P (m):</label>
                    <input type="number" step="any" name="weir_height" value="0.2" required>
                </div>
                <button type="submit">Calculate</button>
            </form>
        </div>

        <!-- Sluice Gate Form -->
        <div id="gateForm" class="form-section">
            <button class="back-btn" onclick="showSelection()">← Back to Selection</button>
            <h2 style="margin-bottom: 20px; color: #333;">Sluice Gate Analysis</h2>
            <div class="info-text">
                Calculate discharge under a sluice gate given upstream and downstream depths.
            </div>
            <form action="/calculate/gate" method="post">
                <div class="form-group">
                    <label>Channel Width (m):</label>
                    <input type="number" step="any" name="width" value="3.0" required>
                </div>
                <div class="form-group">
                    <label>Upstream Depth (m):</label>
                    <input type="number" step="any" name="y_upstream" value="2.0" required>
                </div>
                <div class="form-group">
                    <label>Downstream Depth (m):</label>
                    <input type="number" step="any" name="y_downstream" value="0.3" required>
                </div>
                <button type="submit">Calculate</button>
            </form>
        </div>

        <!-- GVF Form -->
        <div id="gvfForm" class="form-section">
            <button class="back-btn" onclick="showSelection()">← Back to Selection</button>
            <h2 style="margin-bottom: 20px; color: #333;">GVF Profile Calculator</h2>
            <div class="info-text">
                Calculate distance between two depths or solve water surface profiles.
            </div>
            <form action="/calculate/gvf" method="post">
                <div class="form-group">
                    <label>Channel Type:</label>
                    <select name="channel_type" onchange="updateGVFParams(this.value)" required>
                        <option value="rectangular">Rectangular</option>
                        <option value="wide">Wide Channel</option>
                    </select>
                </div>
                <div id="gvfChannelParams" class="channel-params">
                    <div class="form-group">
                        <label>Width (m):</label>
                        <input type="number" step="any" name="width" value="5.0" required>
                    </div>
                </div>
                <div class="form-group">
                    <label>Discharge Q (m³/s):</label>
                    <input type="number" step="any" name="Q" value="7.0" required>
                </div>
                <div class="form-group">
                    <label>Slope S:</label>
                    <input type="number" step="any" name="S" value="0.0006" required>
                </div>
                <div class="form-group">
                    <label>Manning's n:</label>
                    <input type="number" step="any" name="n" value="0.035" required>
                </div>
                <div class="form-group">
                    <label>Start Depth (m):</label>
                    <input type="number" step="any" name="y_start" value="1.0" required>
                </div>
                <div class="form-group">
                    <label>Target Depth (m):</label>
                    <input type="number" step="any" name="y_target" value="2.0" required>
                </div>
                <div class="form-group">
                    <label>Number of Steps:</label>
                    <input type="number" name="num_steps" value="100" required>
                </div>
                <button type="submit">Calculate</button>
            </form>
        </div>
    </div>

    <script>
        function showForm(type) {
            document.getElementById('calcSelection').style.display = 'none';
            document.querySelectorAll('.form-section').forEach(el => el.classList.remove('active'));
            document.getElementById(type + 'Form').classList.add('active');
        }

        function showSelection() {
            document.getElementById('calcSelection').style.display = 'block';
            document.querySelectorAll('.form-section').forEach(el => el.classList.remove('active'));
        }

        function updateChannelParams(channelType) {
            const container = document.getElementById('channelParams');
            let html = '';

            if (channelType === 'rectangular') {
                html = '<div class="form-group"><label>Width (m):</label><input type="number" step="any" name="width" value="5.0" required></div>';
            } else if (channelType === 'trapezoidal') {
                html = '<div class="form-group"><label>Bottom Width (m):</label><input type="number" step="any" name="bottom_width" value="3.0" required></div>';
                html += '<div class="form-group"><label>Side Slope (H:V):</label><input type="number" step="any" name="side_slope" value="1.5" required></div>';
            } else if (channelType === 'circular') {
                html = '<div class="form-group"><label>Diameter (m):</label><input type="number" step="any" name="diameter" value="1.5" required></div>';
            } else if (channelType === 'triangular') {
                html = '<div class="form-group"><label>Side Slope (H:V):</label><input type="number" step="any" name="side_slope" value="1.0" required></div>';
            } else if (channelType === 'wide') {
                html = '<div class="info-text">Wide channel uses unit width (1m)</div>';
            } else if (channelType === 'compound') {
                html = '<div class="form-group"><label>Bottom Section Type:</label>' +
                        '<select name="bottom_type" required>' +
                        '<option value="rectangular">Rectangular</option>' +
                        '<option value="triangular">Triangular</option>' +
                        '</select></div>';
                html += '<div class="form-group"><label>Bottom Width (m):</label><input type="number" step="any" name="bottom_width" value="10.0" required></div>';
                html += '<div class="form-group"><label>Break Depth (m):</label><input type="number" step="any" name="break_depth" value="2.0" required></div>';
                html += '<div class="form-group"><label>Top Section Side Slope (H:V):</label><input type="number" step="any" name="top_side_slope" value="5.0" required></div>';
            }

            container.innerHTML = html;
        }

        function updateJumpParams(channelType) {
            const container = document.getElementById('jumpChannelParams');
            let html = '';

            if (channelType === 'rectangular') {
                html = '<div class="form-group"><label>Width (m):</label><input type="number" step="any" name="width" value="3.0" required></div>';
            } else if (channelType === 'trapezoidal') {
                html = '<div class="form-group"><label>Bottom Width (m):</label><input type="number" step="any" name="bottom_width" value="3.0" required></div>';
                html += '<div class="form-group"><label>Side Slope (H:V):</label><input type="number" step="any" name="side_slope" value="1.5" required></div>';
            } else if (channelType === 'triangular') {
                html = '<div class="form-group"><label>Side Slope (H:V):</label><input type="number" step="any" name="side_slope" value="1.0" required></div>';
            }

            container.innerHTML = html;
        }

        function updateWeirParams(channelType) {
            const container = document.getElementById('weirChannelParams');
            let html = '';

            if (channelType === 'rectangular') {
                html = '<div class="form-group"><label>Width (m):</label><input type="number" step="any" name="width" value="3.0" required></div>';
            } else if (channelType === 'wide') {
                html = '<div class="info-text">Wide channel uses unit width (1m)</div>';
            }

            container.innerHTML = html;
        }

         function updateGVFParams(channelType) {
            const container = document.getElementById('gvfChannelParams');
            let html = '';

            if (channelType === 'rectangular') {
                html = '<div class="form-group"><label>Width (m):</label><input type="number" step="any" name="width" value="5.0" required></div>';
            } else if (channelType === 'wide') {
                html = '<div class="info-text">Wide channel uses unit width (1m)</div>';
            }

            container.innerHTML = html;
        }

    </script>
</body>
</html>
"""

# Results Template
RESULTS_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Calculation Results</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }

        .container {
            max-width: 900px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            padding: 40px;
        }

        h1 {
            color: #667eea;
            text-align: center;
            margin-bottom: 30px;
            font-size: 2.5em;
        }

        .results-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }

        .result-card {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 25px;
            border-radius: 15px;
            box-shadow: 0 5px 15px rgba(0,0,0,0.2);
        }

        .result-label {
            font-size: 0.9em;
            opacity: 0.9;
            margin-bottom: 8px;
        }

        .result-value {
            font-size: 1.8em;
            font-weight: bold;
        }

        .result-unit {
            font-size: 0.8em;
            opacity: 0.8;
        }

        .input-summary {
            background: #f0f4ff;
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 30px;
        }

        .input-summary h3 {
            color: #667eea;
            margin-bottom: 15px;
        }

        .input-item {
            display: flex;
            justify-content: space-between;
            padding: 8px 0;
            border-bottom: 1px solid #e0e0e0;
        }

        .input-item:last-child {
            border-bottom: none;
        }

        button {
            width: 100%;
            padding: 15px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 8px;
            font-size: 18px;
            font-weight: 600;
            cursor: pointer;
            transition: transform 0.2s;
        }

        button:hover {
            transform: scale(1.02);
        }

        .status-badge {
            display: inline-block;
            padding: 5px 15px;
            border-radius: 20px;
            font-size: 0.9em;
            font-weight: 600;
            margin-top: 10px;
        }

        .badge-subcritical {
            background: #4caf50;
            color: white;
        }

        .badge-supercritical {
            background: #f44336;
            color: white;
        }

        .badge-critical {
            background: #ff9800;
            color: white;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>✅ Calculation Results</h1>

        <div class="input-summary">
            <h3>Input Parameters</h3>
            {% for key, value in inputs.items() %}
            <div class="input-item">
                <span><strong>{{ key }}:</strong></span>
                <span>{{ value }}</span>
            </div>
            {% endfor %}
        </div>

        <div class="results-grid">
            {% for key, value in results.items() %}
            <div class="result-card">
                <div class="result-label">{{ key }}</div>
                <div class="result-value">
                    {{ "%.4f"|format(value) if value is number else value }}
                </div>
            </div>
            {% endfor %}
        </div>

        <button onclick="window.location.href='/'">← Calculate Another Problem</button>
    </div>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HOME_TEMPLATE)

@app.route('/calculate/normal', methods=['POST'])
def calculate_normal():
    try:
        # Get form data
        channel_type = request.form['channel_type']
        Q = float(request.form['Q'])
        S = float(request.form['S'])
        n = float(request.form['n'])

        # Build channel parameters
        channel_params = {}
        if channel_type == 'rectangular':
            channel_params['width'] = float(request.form['width'])
        elif channel_type == 'trapezoidal':
            channel_params['bottom_width'] = float(request.form['bottom_width'])
            channel_params['side_slope'] = float(request.form['side_slope'])
        elif channel_type == 'circular':
            channel_params['diameter'] = float(request.form['diameter'])
        elif channel_type == 'triangular':
            channel_params['side_slope'] = float(request.form['side_slope'])

        # Create channel
        if channel_type == 'rectangular':
            channel = RectangularChannel(channel_params['width'])
        elif channel_type == 'trapezoidal':
            channel = TrapezoidalChannel(channel_params['bottom_width'], channel_params['side_slope'])
        elif channel_type == 'circular':
            channel = CircularChannel(channel_params['diameter'])
        elif channel_type == 'triangular':
            channel = TriangularChannel(channel_params['side_slope'])
        elif channel_type == 'wide':
            channel = WideChannel()
        elif channel_type == 'compound':
            bottom_type = request.form['bottom_type']
            bottom_width = float(request.form['bottom_width'])
            break_depth = float(request.form['break_depth'])
            top_side_slope = float(request.form['top_side_slope'])

            channel = CompoundChannel(
                bottom_section_type=bottom_type,
                bottom_params={'width': bottom_width} if bottom_type == 'rectangular' else {'side_slope': 1.0},
                break_depth=break_depth,
                top_section_type='trapezoidal',
                top_params={'bottom_width': bottom_width, 'side_slope': top_side_slope}
            )

            channel_params['bottom_type'] = bottom_type
            channel_params['bottom_width'] = bottom_width
            channel_params['break_depth'] = break_depth
            channel_params['top_side_slope'] = top_side_slope

        # Calculate results
        y_n = normal_depth_manning(channel, Q, S, n)
        y_c = critical_depth(channel, Q)
        Fr = froude_number(channel, y_n, Q)
        regime = flow_regime(Fr)
        S_c = critical_slope_manning(channel, Q, n, y_c)

        # Prepare inputs
        inputs = {
            'Channel Type': channel_type.capitalize(),
            'Discharge (Q)': f"{Q} m³/s",
            'Slope (S)': f"{S}",
            'Manning\'s n': f"{n}"
        }
        inputs.update({k.replace('_', ' ').title(): f"{v}" for k, v in channel_params.items()})

        # Prepare results
        results = {
            'Normal Depth (yₙ)': y_n,
            'Critical Depth (yc)': y_c,
            'Froude Number (Fr)': Fr,
            'Flow Regime': regime.capitalize(),
            'Critical Slope (Sc)': S_c,
            'Area at Normal Depth': channel.area(y_n),
            'Velocity': Q / channel.area(y_n),
            'Hydraulic Radius': channel.hydraulic_radius(y_n)
        }

        return render_template_string(RESULTS_TEMPLATE, inputs=inputs, results=results)

    except Exception as e:
        return f"<h1>Error</h1><p>{str(e)}</p><a href='/'>Go Back</a>"

@app.route('/calculate/jump', methods=['POST'])
def calculate_jump():
    try:
        channel_type = request.form['channel_type']
        Q = float(request.form['Q'])
        y1 = float(request.form['y1'])

        # Build channel parameters
        channel_params = {}
        if channel_type == 'rectangular':
            channel_params['width'] = float(request.form['width'])
            channel = RectangularChannel(channel_params['width'])
        elif channel_type == 'trapezoidal':
            channel_params['bottom_width'] = float(request.form['bottom_width'])
            channel_params['side_slope'] = float(request.form['side_slope'])
            channel = TrapezoidalChannel(channel_params['bottom_width'], channel_params['side_slope'])
        elif channel_type == 'triangular':
            channel_params['side_slope'] = float(request.form['side_slope'])
            channel = TriangularChannel(channel_params['side_slope'])

        # Calculate
        y2 = sequent_depth(channel, y1, Q)
        Fr1 = froude_number(channel, y1, Q)
        Fr2 = froude_number(channel, y2, Q)
        dE = energy_loss_jump(channel, y1, y2, Q)

        inputs = {
            'Channel Type': channel_type.capitalize(),
            'Discharge (Q)': f"{Q} m³/s",
            'Upstream Depth (y₁)': f"{y1} m"
        }
        inputs.update({k.replace('_', ' ').title(): f"{v}" for k, v in channel_params.items()})

        results = {
            'Sequent Depth (y₂)': y2,
            'Froude Number (Fr₁)': Fr1,
            'Froude Number (Fr₂)': Fr2,
            'Energy Loss (ΔE)': dE,
            'Depth Ratio (y₂/y₁)': y2/y1,
            'Upstream Velocity': Q / channel.area(y1),
            'Downstream Velocity': Q / channel.area(y2)
        }

        return render_template_string(RESULTS_TEMPLATE, inputs=inputs, results=results)

    except Exception as e:
        return f"<h1>Error</h1><p>{str(e)}</p><a href='/'>Go Back</a>"

@app.route('/calculate/weir', methods=['POST'])
def calculate_weir():
    try:
        channel_type = request.form['channel_type']
        Q = float(request.form['Q'])
        weir_height = float(request.form['weir_height'])

        if channel_type == 'rectangular':
            width = float(request.form['width'])
            channel = RectangularChannel(width)
        else:
            channel = WideChannel()

        y_upstream = weir_upstream_depth(channel, Q, weir_height)
        y_c = critical_depth(channel, Q)

        inputs = {
            'Channel Type': channel_type.capitalize(),
            'Discharge (Q)': f"{Q} m³/s",
            'Weir Height (P)': f"{weir_height} m"
        }

        if channel_type == 'rectangular':
            inputs['Width'] = f"{width} m"

        results = {
            'Upstream Depth': y_upstream,
            'Critical Depth': y_c,
            'Depth Over Crest': y_c,
            'Specific Energy Upstream': specific_energy(channel, y_upstream, Q),
            'Head Over Weir': y_upstream - weir_height
        }

        return render_template_string(RESULTS_TEMPLATE, inputs=inputs, results=results)

    except Exception as e:
        return f"<h1>Error</h1><p>{str(e)}</p><a href='/'>Go Back</a>"

@app.route('/calculate/gate', methods=['POST'])
def calculate_gate():
    try:
        width = float(request.form['width'])
        y_upstream = float(request.form['y_upstream'])
        y_downstream = float(request.form['y_downstream'])

        channel = RectangularChannel(width)
        Q = sluice_gate_discharge(channel, y_upstream, y_downstream)
        F = sluice_gate_force(channel, y_upstream, y_downstream, Q)

        inputs = {
            'Channel Width': f"{width} m",
            'Upstream Depth': f"{y_upstream} m",
            'Downstream Depth': f"{y_downstream} m"
        }

        results = {
            'Discharge (Q)': Q,
            'Force on Gate': F / 1000,  # Convert to kN
            'Upstream Velocity': Q / channel.area(y_upstream),
            'Downstream Velocity': Q / channel.area(y_downstream),
            'Contraction Ratio': y_downstream / y_upstream
        }

        return render_template_string(RESULTS_TEMPLATE, inputs=inputs, results=results)

    except Exception as e:
        return f"<h1>Error</h1><p>{str(e)}</p><a href='/'>Go Back</a>"

@app.route('/calculate/gvf', methods=['POST'])
def calculate_gvf():
    try:
        channel_type = request.form['channel_type']
        Q = float(request.form['Q'])
        S = float(request.form['S'])
        n = float(request.form['n'])
        y_start = float(request.form['y_start'])
        y_target = float(request.form['y_target'])
        num_steps = int(request.form['num_steps'])

        if channel_type == 'rectangular':
            width = float(request.form['width'])
            channel = RectangularChannel(width)
        else:
            channel = WideChannel()

        # Calculate
        y_n = normal_depth_manning(channel, Q, S, n)
        y_c = critical_depth(channel, Q)
        S_c = critical_slope_manning(channel, Q, n, y_c)
        slope_class = slope_classification(S, S_c)
        distance = distance_to_depth_manning(channel, Q, S, n, y_start, y_target, num_steps)

        inputs = {
            'Channel Type': channel_type.capitalize(),
            'Discharge (Q)': f"{Q} m³/s",
            'Slope (S)': f"{S}",
            'Manning\'s n': f"{n}",
            'Start Depth': f"{y_start} m",
            'Target Depth': f"{y_target} m"
        }

        if channel_type == 'rectangular':
            inputs['Width'] = f"{width} m"

        results = {
            'Distance': distance,
            'Normal Depth': y_n,
            'Critical Depth': y_c,
            'Critical Slope': S_c,
            'Slope Classification': slope_class
        }

        return render_template_string(RESULTS_TEMPLATE, inputs=inputs, results=results)

    except Exception as e:
        return f"<h1>Error</h1><p>{str(e)}</p><a href='/'>Go Back</a>"




print("Flask app created successfully!")

Flask app created successfully!


### Step 3: Run the Web Interface

Run these cells to start the web server. You'll get a public URL that you can click to access your calculator!

In [25]:
%%bash
ngrok config add-authtoken "34QMyaEcJmlspuvD4RC1RZs8nEI_6x83hjEr3xa8fG3thQTA1"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [26]:
from pyngrok import ngrok
ngrok.kill()
print("All ngrok processes killed. Wait 5 seconds before starting your app.")

All ngrok processes killed. Wait 5 seconds before starting your app.


In [27]:
# from pyngrok import ngrok
# import threading
# import time

# # Terminate any existing ngrok tunnels
# ngrok.kill()
# time.sleep(2) # Give a moment for processes to terminate

# # Set up ngrok to create a public URL
# # Using a dynamic port might also help avoid conflicts if the default is somehow stuck
# public_url = ngrok.connect(5000)
# print("="*70)
# print("🌊 OPEN CHANNEL FLOW CALCULATOR IS RUNNING! 🌊")
# print("="*70)
# print(f"\n✅ Click this link to access your calculator:")
# print(f"   {public_url}")
# print(f"\n📱 You can also access it from your phone using the same link!")
# print("\n⚠️  Keep this cell running to keep the web app active.")
# print("="*70)

# # Run Flask app in a separate thread
# def run_app():
#     # Use debug=False in production
#     app.run(port=5000, debug=False, use_reloader=False)

# thread = threading.Thread(target=run_app)
# thread.start()

# # Keep the main thread alive to allow the Flask thread to run
# try:
#     while True:
#         time.sleep(1)
# except KeyboardInterrupt:
#     print("\nShutting down server...")
#     ngrok.kill()
#     thread.join()
#     print("Server shut down.")



from pyngrok import ngrok
import threading
import time

# More thorough cleanup
try:
  # Get all active tunnels and disconnect them
  tunnels = ngrok.get_tunnels()
  for tunnel in tunnels:
      print(f"Closing tunnel: {tunnel.public_url}")
      ngrok.disconnect(tunnel.public_url)
  time.sleep(1)
except Exception as e:
  print(f"Cleanup note: {e}")

# Kill the ngrok process
ngrok.kill()
time.sleep(3)  # Increased wait time

# Set up ngrok to create a public URL
public_url = ngrok.connect(5000)
print("="*70)
print("🌊 OPEN CHANNEL FLOW CALCULATOR IS RUNNING! 🌊")
print("="*70)
print(f"\n✅ Click this link to access your calculator:")
print(f"   {public_url}")
print(f"\n📱 You can also access it from your phone using the same link!")
print("\n⚠️  Keep this cell running to keep the web app active.")
print("="*70)

# Run Flask app in a separate thread
def run_app():
  app.run(port=5000, debug=False, use_reloader=False)

thread = threading.Thread(target=run_app)
thread.start()

# Keep the main thread alive
try:
  while True:
      time.sleep(1)
except KeyboardInterrupt:
  print("\nShutting down server...")
  ngrok.kill()
  thread.join()
  print("Server shut down.")

🌊 OPEN CHANNEL FLOW CALCULATOR IS RUNNING! 🌊

✅ Click this link to access your calculator:
   NgrokTunnel: "https://nonbearded-osculant-camellia.ngrok-free.dev" -> "http://localhost:5000"

📱 You can also access it from your phone using the same link!

⚠️  Keep this cell running to keep the web app active.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Oct/2025 16:59:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2025 16:59:56] "POST /calculate/normal HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2025 17:00:04] "GET / HTTP/1.1" 200 -



Shutting down server...


KeyboardInterrupt: 